In [1]:
import requests
from bs4 import BeautifulSoup
import re

# Possible features to add?
* Number of employees at brewery
* Numbers of beers by brewery
* Age of brewery
* Economic climate surrounding brewery locations (average incomes)
* Number of locations for brewery
* I feel like I need something closer to amounts of:
    * Hops
    * wheat
    * barly
    * water mineral content

# Accessing the page

Planning on doing everything form the beer styles page as that seems to be the easiest page from which to access all beers.

In [2]:
url = 'https://www.beeradvocate.com/beer/styles/'

response = requests.get(url)

In [3]:
response.status_code # status code = 200 => OK

200

In [4]:
# print(response.text)

In [5]:
page = response.text

In [6]:
soup = BeautifulSoup(page, "lxml")

In [7]:
# print(soup.prettify())

# Creating Styles Lists

In [8]:
styles_links = [link for link in soup.find_all('a') if 'beer/styles' in str(link)]

In [9]:
styles_links[0]

<a href="/beer/styles/">Beer Styles</a>

### Styles Numbers

In [10]:
styles_nums = []

for link in styles_links[2:]:
    styles_nums.append(str(link).split('/')[3])

In [11]:
sorted(styles_nums)[:5]

['#XenForo"><i class="uix_icon uix_icon-jumpToTop"><',
 '10',
 '101',
 '11',
 '114']

In [12]:
styles_nums = sorted(styles_nums)[1:]

In [13]:
styles_nums[0]

'10'

### Styles Names

In [14]:
styles_names = []

for link in styles_links[2:]:
    style_name = re.split(r'>|<', str(link))[2]
    styles_names.append(style_name)

In [15]:
styles_names[0]

'German Bock'

In [16]:
sorted(styles_names)[:5]

['',
 'American Adjunct Lager',
 'American Amber / Red Ale',
 'American Amber / Red Lager',
 'American Barleywine']

In [17]:
styles_names = sorted(styles_names)[1:]

In [18]:
styles_names[0]

'American Adjunct Lager'

## Creating links to Beer Style Pages

Realized that the links don't have descriptions, rather a range of values from ~5 - ~190

Using range: *Note that to use this set you'd need to write a pass in the event that the page failed/didn't load*

In [19]:
url = 'https://www.beeradvocate.com/beer/styles/'

styles_links = []

for num in range(5,190):
    styles_links.append(url+str(num))

Using list of nums

In [20]:
url = 'https://www.beeradvocate.com/beer/styles/'

styles_links = []

for num in styles_nums:
    styles_links.append(url+num)

# Selecting Beer from Beer Styles Pages

In [21]:
url = styles_links[0]

response = requests.get(url)

In [22]:
response.status_code # status code = 200 => OK

200

In [23]:
# print(response.text)

In [24]:
page = response.text

In [25]:
soup = BeautifulSoup(page, "lxml")

In [26]:
# print(soup.prettify())

In [27]:
beer_links = [link for link in soup.find_all('a') if '/beer/profile/' in str(link)]

### Beer Numbers

Note that the beer numbers also need to include the brewery number to properly create a link

In [54]:
beer_nums = []

for link in beer_links:
    beer_num = str(link).split('/')
    if len(beer_num) > 6:
        
        # take both brewering number and beer number
        
        brew, beer = beer_num[3:5]
        brew_beer = brew + '/' + beer + '/'
        beer_nums.append(brew_beer)
    else:
        pass
    

In [55]:
beer_nums[0]

'/388/5281/'

In [36]:
len(beer_nums)

50

### Beer Names

In [30]:
beer_names = []

for link in beer_links:
    beer_name = re.split(r'>|<', str(link))
    if len(beer_name) > 6:
        beer_names.append(beer_name[4])
    else:
        pass

In [31]:
beer_names[0]

"Cantillon Fou' Foune"

In [37]:
len(beer_names)

50

In [32]:
sorted(beer_names)[:5]

['Belle Vue Kriek',
 'Blackberry ',
 'Boon Framboise',
 'Boon Kriek',
 'Boon Oude Kriek Mariage Parfait']

# Getting Beer Info

In [69]:
response = requests.get('https://www.beeradvocate.com/beer/profile/388/5281/')
page = response.text
soup = BeautifulSoup(page, "lxml")

### Name and Brewery

In [262]:
title_div = soup.find(class_='titleBar')

In [266]:
beer_brew_names = str(title_div.text).replace('\n','')
beer_brew_names

"Cantillon Fou' Foune | Brasserie Cantillon"

### Score

In [89]:
score_obj = soup.find('span', {'class': 'BAscore_big'})

In [90]:
score = float(re.split(r'>|<', str(score_obj))[4])
score

4.65

### Class

In [91]:
beer_class_obj = soup.find_all('b')[4]

In [96]:
beer_class = re.split(r'>|<', str(class_obj))[2]
beer_class

'World-Class'

### Number of ratings

In [98]:
ratings_obj = soup.find('span', {"class": "ba-ratings"})

In [110]:
ratings = int(re.split(r'>|<', str(ratings_obj))[2].replace(',',''))
ratings

3079

### Beer Stats

In [205]:
stats_objs = soup.find_all('dd')[5:13]
stats_objs[0]

<dd>#23</dd>

In [175]:
stats_list = []

for item in stats_objs:
    item = str(item.text).strip()
    if len(item) > 0:
        clean_item = re.sub(r'#|%|,','',item)
        
        if '.' in clean_item:
            stats_list.append(float(clean_item))
        else:
            stats_list.append(int(clean_item))
    else:
        pass
    
    stats_list

In [178]:
ranking, reviews, ratings, pDev, wants, gots, trade = stats_list

In [180]:
ranking, reviews, ratings, pDev, wants, gots, trade 

(23, 562, 3079, 7.31, 2811, 669, 58)

### Brewery and Region

In [224]:
brew_objs = soup.find_all('a')[119:123]
brew_objs[0]

<a href="/beer/profile/388/"><b>Brasserie Cantillon</b></a>

In [203]:
brew_objs_list = []

for obj in brew_objs:
    brew_objs_list.append(obj.text)

In [206]:
brew, region, site, style = brew_objs_list

In [207]:
brew, region, site, style 

('Brasserie Cantillon', 'Belgium', 'cantillon.be', 'Belgian Fruit Lambic')

### ABV

In [230]:
abv_div = soup.find('div',attrs={'id':'info_box'})
abv_div

<div class="break" id="info_box" style="float:right;width:70%;">
<div id="main_pic_norm" style="text-align:center; float:right; width:150px; padding:0px; margin:0px 5px 25px 25px;"><div style="position:relative;"><img alt="Cantillon Fou' Foune" border="0" height="300" src="https://cdn.beeradvocate.com/im/beers/5281.jpg" width="90"/><span style="position:absolute;left:0;top:0;"><img alt="Cantillon Fou' Foune" border="0" height="300" src="https://cdn.beeradvocate.com/im/c_beer_image.gif" width="150"/></span></div></div>
<b>BEER INFO</b>
<br/><br/>
<b>Brewed by:</b>
<br/>
<a href="/beer/profile/388/"><b>Brasserie Cantillon</b></a>
<br/><a href="/place/directory/6/BE/">Belgium</a><br/><a href="http://cantillon.be" target="_blank">cantillon.be</a> <br/><br/>
<b>Style:</b> <a href="/beer/styles/10/"><b>Belgian Fruit Lambic</b></a>
<br/><br/>
<b>Alcohol by volume (ABV):</b> 5.00%
<br/><br/>
<b>Availability:</b> Rotating
<br/><br/>
<b>Notes / Commercial Description:</b>
<br/>
Apricot Lambic<br

In [245]:
abv = float(str(abv_div).split('(ABV):</b>')[1][1:5])
abv

5.0

### Availability
Note that we simply reuse the ABV div

In [244]:
availability_str = str(abv_div).split('Availability:</b> ')[1]
availability = availability_str.split('\n')[0]
availability

'Rotating'

### Commercial Description
Note that we simply reuse the ABV div

In [251]:
comm_desc_str = str(abv_div).split('Description:</b>\n<br/>\n')[1]
comm_desc = comm_desc_str.split('<br')[0]
comm_desc

'Apricot Lambic'

### Date Added
Note that we simply reuse the ABV div

In [259]:
date_added_str = str(abv_div).split('<br/><br/>')[-2]
date_added = date_added_str.split()[-1]
date_added

'08-05-2002'

### Adding sleep

In [272]:
import time
import numpy as np

In [324]:
time.sleep(np.random.poisson(10)/10)

print('hey')

hey


1.2

# Following Links!

### Psuedo code

``` python
for link in style_links:
    
    get_beer_links()
    
    for link in beer_links:
        
        follow_beer_link_get_data()
```
        

# Everything below here may be out of date. See Implementing_Scraping.ipynb

### Let's try real code!

In [38]:
def get_style_links():
    # we start on the styles page because it is the easiest page 
    # from which to access all beers
    
    followable_links = []
    url = 'https://www.beeradvocate.com/beer/styles/'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    # get style links

    styles_links = [link for link in soup.find_all('a') if 'beer/styles' in str(link)]

    styles_nums = []

    for link in styles_links[2:]:
        styles_nums.append(str(link).split('/')[3])
        
    styles_nums = sorted(styles_nums)[1:]
    
    for num in styles_nums:
        followable_links.append(url + num)
    
    return followable_links

In [59]:
style_links = get_style_links()
style_links[0]

'https://www.beeradvocate.com/beer/styles/10'

In [61]:
def get_brew_beer_links(url):
    
    followable_links = []
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    beer_links = [link for link in soup.find_all('a') if '/beer/profile/' in str(link)]
    
    beer_nums = []

    for link in beer_links:
        beer_num = str(link).split('/')
        if len(beer_num) > 6:

            # take both brewering number and beer number

            brew, beer = beer_num[3:5]
            brew_beer = brew + '/' + beer + '/'
            beer_nums.append(brew_beer)
        else:
            pass
    
    prof_url = 'https://www.beeradvocate.com/beer/profile/'
    
    for num in beer_nums:
        followable_links.append(prof_url + num)
    
    return followable_links
    

In [62]:
brew_beer_link = get_brew_beer_links(style_links[0])
brew_beer_link[0]

'https://www.beeradvocate.com/beer/profile/388/5281/'

In [66]:
def get_beer_info(url):
    
    out = []
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    
    
    print(soup)

In [68]:
beer_info = get_beer_info(brew_beer_link[0])
beer_info

In [ ]:
def get_beer_info():
    
    style_links = get_style_links()
    
    for style_link in style_links:
        
        brew_beer_links = get_brew_beer_links(style_link)
        
        for brew_beer_link in brew_beer_links:
            
            get_beer_info(brew_beer_link)
            
            get_brew_info(brew_beer_link)
    
    
    
    

# To Do Next

* Be able to select the next 50 pages
* be able to select the table of information from each page
* Get list of brewery names/numbers
* be able to select information on each brewery

# Thoughts / Notes

Just realized that each beer is within each beer profile
* Let's say the beer profile (brewery) is 388 and the beer number is 5281
* This would make the url end with /388/5281/
* That means that we should be able to go through a range of values for the brewery and beers and simply skip those that don't work
* Problem with this is that it might look like a DDOS... we could hit the page easily (brewery) 400 * (beer) 6000 = 2,400,000 times...
* It would actually be way higher... Highest beer num is around 25,000